# 🔍 PubMed Search and Embedding Workflow

This notebook demonstrates a pipeline to search PubMed, download relevant documents, process PDFs, generate embeddings using OpenAI, and store them in Pinecone.

In [1]:
# Standard Library
import os
import gc
import subprocess

# Third-party Libraries
import pandas as pd
from tqdm import tqdm
from Bio import Entrez
import pubchempy as pcp
from openai import OpenAI

from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

from langchain_pinecone import PineconeVectorStore
from rich import print

# Local Modules
from demo_script import (
    search_n_download,
    run_bash_script,
    pubmed_search_get_id,
    process_pdf,
    create_embeddings,
    upsert_embeddings
)


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load API Keys from Environment Variables

client = OpenAI(api_key="sk-proj-T0Ocs*********aPDU9DATXAA")
MODEL = "text-embedding-ada-002"
pc = Pinecone(api_key="pcsk_4bbPdM_M8M*********xyzg1y4DYfi36SaSs6TTk8sQB")


In [78]:
smile_list = pd.read_csv("./DIA_trainingset_RDKit_descriptors.csv")

In [79]:
list(smile_list["SMILES"][1:5])

['C[C@H](N(O)C(=O)N)c1cc2ccccc2s1',
 'C[N+](C)(C)CC(=O)[O-]',
 'CC(C)n1c(\\C=C\\[C@H](O)C[C@H](O)CC(=O)O)c(c2ccc(F)cc2)c3ccccc13',
 'C\\C(=C(\\C#N)/C(=O)Nc1ccc(cc1)C(F)(F)F)\\O']

In [ ]:
email = 'your_email@mail.com'
script_path = './down_pdf.sh' 
index_name = "hs-codes3" ## Change this to your desired index name
folder = "./research_papers"


if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )


index = pc.Index(index_name)

for i in tqdm(list(smile_list["SMILES"][1:5])):

    search_n_download(i,email,script_path,10)  ### takes percent amount of the search results to download

    pdf_files = [f for f in os.listdir(folder) if f.endswith('.pdf')]
    
    for file in tqdm(pdf_files):
        file_path = "./research_papers" + file

        if os.path.getsize(file_path) == 0:
            print(f"File is empty: {file_path}")
            pass
        else:
            paper_name_tag = os.path.splitext(os.path.basename(file_path))[0]
            # print(file_path)
            texts = process_pdf(file_path)
            embeddings = create_embeddings(texts)
            upsert_embeddings(index, embeddings, texts, name_integ = paper_name_tag)

        
        os.remove(file_path)
        gc.collect()



## Now you have a pinecone index where you can search docs based on your query and also build chatbot that provides references based on your collected papers


In [ ]:

# Get openai api key from platform.openai.com
OPENAI_API_KEY = "sk-proj-T0Ocs*********OcPaPDU9DATXAA"

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [ ]:


pinecone_vectorstore = PineconeVectorStore(
    index_name=index_name, 
    embedding=embed, 
    text_key="text",
    pinecone_api_key = "pcsk_4bbPdM_M8MnFKNao*********iyByxyzg1y4DYfi36SaSs6TTk8sQB"
)

In [71]:


documents = pinecone_vectorstore.similarity_search(
    query="Any interesting finding on betaine ?,cite the sources and pinecone database paper too",  # our search query
    k=5  # return 3 most relevant docs
)

for doc in documents:
    print(doc.__dict__)
    print()

{
    'id': '16662637chunk-26',
    'metadata': {},
    'page_content': 'BETAINEINBETASPP.\n120\n80k\n40\nN\n30\n20_\n10o_\n0\nII\nB.LEAF\n 
BETAINE\n--a--PHOSPHATIDYL-CHOLINE\n *\\ CHOLINE\n A PHOSPHORYL-\nCHOLINE\nBETAINE\n0 2 0 
2\nINCUBATIONTIME(h)\nFIG.3.TimecoursesforconversionofIjImolof["4C]ethanolamineto\nbetaineandotherQACsbytaprootsink
s(A)andleaves(B)ofintact\nGreatWesternD-2sugarbeetplants,salinizedwith150mmNaCl.Plants\nwerefed[\'4Clethanolamine(1
.40,uCijimol-I)viathetaprootsinkand\nincubatedindarkness,orwerefed["4Clethanolamine(3.29,uCi,tmol-\')\nviaaleafblad
eandincubatedinthelight.Eachdatapointisthemeanfor\nthreeplants.Theconversionof[\'4C]ethanolaminetolabeledproductsis
\nexpressedinnmol(calculatedfromthespecificactivityoftheethanolamine\nsupplied)tofacilitatecomparisonofrootandleafl
abelingdata.Thefed\nleafandthetaprootsinkweresimilarindryweight(leaf= 87mg,taproot\n+hypocotyl= 
97mg;meansforthreeorgans).Theexperimentswere\nrepeated,analyzingonly[\'4C]betaineand["C]phosphatidylcholine,with\nv
erysimilarresults.\nSimilarlyclosecorrelationsforsinglegenotypesgrownatvarious',
    'type': 'Document'
}

{
    'id': '39675709chunk-70',
    'metadata': {},
    'page_content': '11. Kärkkäinen, O., Tuomainen, T., Koistinen, V., Tuomainen, M., Lep-\npänen, J., Laitinen, 
T.,et al. (2018) Whole grain intake associated mole-\ncule 5-aminovaleric acid betaine decreasesb-oxidation of 
fatty acids in\nmouse cardiomyocytes. Sci. Rep. 8, 13036\n12. Lindstedt, G., and Lindstedt, S. (1970) Cofactor 
requirements of g-\nbutyrobetaine hydroxylase from rat liver.J. Biol. Chem.245, 4178–4186\n13. Vaz, F. M., and 
Wanders, R. J. A. (2002) Carnitine biosynthesis in\nmammals. Biochem. J. 361, 417–429\n14. Yasumoto, T., and 
Shimizu, N. (1977) Identi ﬁcation of delta-valer-\nobetaine and other betaines in the ovary of shellﬁsh Callista 
brevishipo-\nnata. Bull. Jpn. Soc. Sci. Fish43, 201–206\n15. Keiser, M. J., Roth, B. L., Armbruster, B. N., 
Ernsberger, P., Irwin, J. J., and\nShoichet, B. K. (2007) Relating protein pharmacology by ligand chemistry.\nNat. 
Biotechnol. 25, 197–206\n16. Tars, K., Leitans, J., Kazaks, A., Zelencova, D., Liepinsh, E., Kuka, J.,et al.',
    'type': 'Document'
}

{
    'id': '26861782chunk-46',
    'metadata': {},
    'page_content': 'to implementing long-term trials that test the effect of\ndietary betaine supplementation in 
individuals at high\nrisk of diabetes.\nThe limitations of our study deserve comment. First,\nwe measured a subset 
of 84 circulating metabolites out of\nthe estimated 5,000– 10,000 species within the metabo-\nlome. Our results are
thus agnostic to the predictive ca-\npacity of metabolites not measured on our platform\n(26,27). Utilization of 
novel, nontargeted measures of\nmetabolites may highlight additional markers of diabetes\nrisk within this 
population. Although our method is ro-\nbust to the measurement of small organic amines, alterna-\ntive methods of 
metabolite proﬁling, speciﬁcally nuclear\nmagnetic resonance, could be used to extend ourﬁndings.\nSecond, samples 
for direct measurement were selected\nbased on a nested case-control design and established sta-\ntistical methods 
were used to extend the results to the\nentire DPP. This approach was chosen given logistical is-',
    'type': 'Document'
}

{
    'id': '16662637chunk-41',
    'metadata': {},
    'page_content': 
'wholemustberegulatedprincipallyviatheactivityofthesynthesis\npathwayintheleaves.\nBetaineisaccumulatedconstitutive
lyinunsalinizedbeetplants\nandsalinizationinducesfurtheraccumulation;domesticationhas\nnotgreatlyalteredthis.Howeve
r,withintheprimarygenepoolof\nsugarbeettheremaybetwosortsofgeneticvariationforbetaine:\n(a)variationforthebetaineac
cumulatedbytheplantasawhole\n(presumablyduetovariationintherateofsynthesisbyleaves);\n(b)variationforbetainepartiti
oningtotheroot(presumably\narisingfromvariationintheexportofbetainefromleaves).\nAcknowledgments-WethankKimberlyDit
zandMargaretHoldredgefortech-\nnicalassistance,andDr.J.A.D.Zeevaartforadviceuponphotoperiodexperiments.\nLITERATURE
CITED\n1.BOWMANMS,RROHRINGER1970Formatemetabolismandbetaineformation\ninhealthyandrust-infectedwheat.CanJBot48:803-
811\n2.CAMPBELLGKG1976Sugarbeet.InNWSimmonds,ed,EvolutionofCrop\nPlants.Longman,London,pp25-28\n3.COUGHLANSJ,RGWYNJ
ONES1980SomeresponsesofSpinaceaoleraceatosalt\n1197',
    'type': 'Document'
}

{
    'id': '31848620chunk-20',
    'metadata': {},
    'page_content': 'metabolite correlations. Of these, only 5 to our 
knowledge\nhavebeenpreviouslyreportedinpopulationandfeedingstudies\n(proline betaine and citrus, 1-methylhistidine 
and chicken,\nand pantothenic acid, thiamin, 5-methyltetrahydrofolate, 
and\nsupplements)(6,9,12,22).Ourfindingsaddto thegrowing list\nof candidate dietary biomarkers to be validated and 
potentially\nused in nutritional epidemiology studies.\nTo date, these 2 metabolomics platforms have been used\nin 
2 prior dietary studies (14, 15), both of which focused on\nspecific investigator-assigned diets. In contrast to 
these studies,\nthe present study covers a broad span of different aspects of\ndiet, which likely explains the 
large number of novel findings.\n696 Mazzilli et al.',
    'type': 'Document'
}

In [ ]:
## Now you can use this vector database to answer queries with citations on sources. 

In [72]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# Chat Completion LLM
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4.5-preview',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=pinecone_vectorstore.as_retriever()
)

In [74]:
answer = qa.invoke("Any interesting finding on betaine ?\
    Cite the sources and are congruent with data from pinecone database")

In [75]:
from rich import print
print(answer['result'])

Yes, there are several interesting findings regarding betaine:

1. Betaine has been shown to accumulate significantly in plants under saline conditions, suggesting a role in 
osmoregulation. Specifically, in sugar beet (Beta spp.), betaine accumulation in response to salt stress could 
significantly contribute to cytoplasmic osmoregulation. This is supported by the hypothesis that betaine is 
compartmentalized primarily in the cytoplasm rather than vacuoles, thus functioning as a cytosolic osmoticum 
(source: provided context).

2. In sugar beet plants exposed to saline conditions (150 mM NaCl), labeled ethanolamine was converted into betaine
and other quaternary ammonium compounds (QACs). This indicates active metabolic pathways converting ethanolamine 
into betaine under stress conditions, highlighting betaine's role in plant stress responses (source: provided 
context).

3. Additionally, betaine-related metabolites have been identified as potential dietary biomarkers in nutritional 
epidemiology studies. For example, proline betaine has been associated with citrus intake, and other betaine 
derivatives have been linked to specific dietary sources, suggesting their potential use as biomarkers for dietary 
intake (source: provided context, Mazzilli et al.).

However, regarding congruency with data from the pinecone database, the provided context does not mention or 
reference the pinecone database explicitly. Therefore, I cannot confirm congruency with data from the pinecone 
database based on the information provided.

In [76]:
answer = qa.invoke("Any interesting finding on cd275 ?\
    Cite the sources and are congruent with data from pinecone database")

print(answer['result'])

Based on the provided context, there is no information available regarding CD275. Therefore, I cannot provide any 
interesting findings or cite sources related to CD275 from the given data.